In [ ]:
import pandas as pd
import numpy as np

#Read in data and set options
d = pd.read_excel("data.xlsx", index_col = 0)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
d.columns = [c.replace(' ', '_') for c in d.columns]

In [ ]:
d.clean = d.copy()

#Drop duplicate and empty columns
d.clean = d.drop(['Location_(Location_Name)', 'Responsibilities_and_Achievements', 'Nationality_(Label)'], axis=1)

In [ ]:
#Create column for whether we have eduaction info
d['Highest_Degree'].fillna("0", inplace = True)

d.clean['Education'] = d.apply(lambda x: 0 if x[list(d.columns).index('Highest_Degree')] == "0" else 1, axis = 1)

#Drop other education fields
d.clean.drop(['Degree_(Picklist_Label)', 'Did_you_graduate?_(Picklist_Label)', 
        'Major_(Picklist_Label)', 'Education_Record_is_blank?', 'Highest_Degree', 
        'Is_Highest_Degree?'], axis=1, inplace = True)

In [ ]:
#Bin marital statuses
d.clean['Marital_Status'] = d.apply(lambda x: "Single" if x[list(d.columns).index('Marital_Status_(Label)')] == "Single" 
                                    else ("Married/Living Together" if x[list(d.columns).index('Marital_Status_(Label)')] in 
                                    ['Married', 'Domestic Partner', 'Domestic Partner (Unregistered Marriage)', 
                                    'Living Together', 'Widowed With Surviving Pension'] else ("Divorced/Widowed" 
                                    if x[list(d.columns).index('Marital_Status_(Label)')] in
                                    ['Divorced', 'Legally Separated', 'Widowed'] else "Unknown")), axis = 1)

#Drop original marital status column
d.clean.drop('Marital_Status_(Label)', axis = 1, inplace = True)

In [ ]:
#Drop some other irrelevant columns
#Chose to drop Job_Category_(Picklist_Label) because it is identical to Job_Category_(Job_Class) with more NA values
d.clean.drop(['Rating_Label', 'Time_Type', 'Generation', 'Is_Fulltime_Employee?', 
              'Frequency', 'Employee_Type', 'Current_Hire_Date', 'Continuous_Service_Date', 
              'Length_of_Service_-_Buckets','Job_Family', 'Location_Name'
              'Comp_Grade_Profile','Currency','Annualized_Mid_Point',
              'Pay_Grade', 'Organization_(Label)','Termination_Date'], axis = 1, inplace = True)

In [ ]:
#Drop interns - at request of company

index = d.clean[d.clean['Employee_Type'] == 'Intern'].index

d.clean.drop(index, inplace = True)

In [ ]:
#Create list of "remote" locations

df = d.clean.loc[d.clean['Location_Name'].str.startswith('Remote') == True]
df = pd.DataFrame(df['Location_Name'].value_counts())
remotes = list(df.index)

#Create list of locations with very few employees
dfNew = pd.DataFrame(d.clean['Location_Name'].value_counts())
smalls = list(dfNew[dfNew['Location_Name'] < 50].index)

#Create New Location Column - using lists above
d.clean['Location'] = d.clean.apply(lambda x: 'Remote' if x[list(d.clean.columns).index('Location_Name')] in remotes 
                                    else ('Other' if ((x[list(d.clean.columns).index('Location_Name')] not in remotes))
                                          & (x[list(d.clean.columns).index('Location_Name')] in smalls) 
                                          else x[list(d.clean.columns).index('Location_Name')]), axis = 1)

In [ ]:
#Drop duplicate rows

d.clean.drop_duplicates(inplace = True)
d.clean.drop(index = 22613, axis = 0, inplace = True)
d.clean.drop(index = 46660, axis = 0, inplace = True)
d.clean.drop(index = 26256, axis = 0, inplace = True)

#Set GID_anonymized as index

d.clean.set_index('GID_anonymized', inplace = True)

In [ ]:
#Fill in null values in the compensation grade column using information from other columns

for i in range(d.clean.shape[0]):
    if ((pd.isna(d.clean['Compensation_Grade'][i])) & (not pd.isna(d.clean['Job_Category'][i])) & (not pd.isna(d.clean['Job_Level'][i]))):
        d.clean['Compensation_Grade'][i] = d.clean['Job_Category'][i] + ' ' + str(int(d.clean['Job_Level'][i]))

In [ ]:
#Fill null vaues in gender column

d.clean['Gender'] = d.clean[['Gender']].fillna('D', axis = 1)

In [ ]:
#Create dataframe that contains the mean count of direct reports by compensation grade (not including outliers)

newDF = d.clean[d.clean['Count_of_Direct_Reports'] > 0]
meanReports = pd.DataFrame(round(newDF.groupby('Compensation_Grade').Count_of_Direct_Reports.mean(), 0))
meanReports.reset_index(level = 0, inplace = True)

#Impute mean values for outliers in direct reports column

for i in range(len(d.clean['Count_of_Direct_Reports'])):
    if d.clean['Count_of_Direct_Reports'][i] < 0:
        comp = d.clean['Compensation_Grade'][i]
        d.clean['Count_of_Direct_Reports'][i] = meanReports[meanReports['Compensation_Grade'] == comp]['Count_of_Direct_Reports']

In [ ]:
#Bin ethnicities
#We realize that binning races together can be potentially color blind but felt we were able to do this in a fair way due to the amazing diversity of our team.


d.clean['Ethnicity'] = d.clean.apply(lambda x: "Native American" if x[list(d.clean.columns).index('Race/Ethnicity')] in
                                     ["American Indian or Alaskan Native, not Hispanic or Latino", "Native Hawaiian or Other Pacific Islander, not Hispanic or Latino"]
                                     else ("White" if x[list(d.clean.columns).index('Race/Ethnicity')] in
                                          ["White, not Hispanic or Latino", "Caucasian", "Eurasian"] else (
                                          "SE Asian" if x[list(d.clean.columns).index('Race/Ethnicity')] in 
                                          ["Malay", "Sarawakian", "Vietnamese", "Filipino"] else (
                                          "South Asian" if x[list(d.clean.columns).index('Race/Ethnicity')] in 
                                          ["Indian", "Sikh", "Pakistani"] else (
                                          "Other/Unknown" if x[list(d.clean.columns).index('Race/Ethnicity')] in
                                          ["I do not wish to disclose", "Two or More Races, not Hispanic or Latino", "others"] 
                                          else x[list(d.clean.columns).index('Race/Ethnicity')] )))), axis = 1)
d.clean['Ethnicity'] = d.clean[['Ethnicity']].fillna('Other/Unknown', axis = 1)

In [ ]:
#Target Variable
#Maps to 1 if event reason name is "Voluntary Termination" or if termination reason in "Job Abandonment"

d.clean['Status'] = d.clean.apply(lambda x: 1 if 
                                  (x[list(d.clean.columns).index('Event_Reason_Name')] == 'Voluntary Termination') | 
                                  ((x[list(d.clean.columns).index('Event_Reason_Name')] == 'Involuntary Termination') &
                                  (x[list(d.clean.columns).index('Termination_Reason_(externalName)')] == 'Job Abandonment'))
                                 else 0, axis = 1)

In [ ]:
#Drop more columns

d.clean.drop(['Employee_Status', 'Event_Reason_Name', 'Termination_Reason_(externalName)',
             'Race/Ethnicity', 'Management_Level_(Picklist_Label)', 'Status'], axis = 1, inplace = True)

In [ ]:
'''
At this point, we had done everything we could to fill the NAs with the information
provided to us by company.  Given time constraints and other limitations, we were 
unable to fill any more in a reasonable way.  Here we create a new dataframe without
any NA values - we did this using .copy() in case this data could be provided later.
'''

d.clean.NoNa = d.clean.copy()
d.clean.NoNa.dropna(inplace = True)

#Make another copy to perform further preprocessing
d.clean.preprocess = d.clean.NoNa.copy()

#Normalize continuous features
d.clean.preprocess['Age'] = (d.clean.preprocess['Age'] - d.clean.preprocess['Age'].mean())/d.clean.preprocess['Age'].std()
d.clean.preprocess['Compa_Ratio'] = (d.clean.preprocess['Compa_Ratio'] - d.clean.preprocess['Compa_Ratio'].mean())/d.clean.preprocess['Compa_Ratio'].std()
d.clean.preprocess['Count_of_Direct_Reports'] = (d.clean.preprocess['Count_of_Direct_Reports'] - d.clean.preprocess['Count_of_Direct_Reports'].mean())/d.clean.preprocess['Count_of_Direct_Reports'].std()
d.clean.preprocess['Years_in_Service_(Continuous_Service_Date)'] = (d.clean.preprocess['Years_in_Service_(Continuous_Service_Date)'] - d.clean.preprocess['Years_in_Service_(Continuous_Service_Date)'].mean())/d.clean.preprocess['Years_in_Service_(Continuous_Service_Date)'].std()

#Factorize categorical features with natural ordinal relationships
d.clean.preprocess['Job_Level'] = pd.factorize(d.clean.preprocess['Job_Level'], sort = True)[0]

d.clean.preprocess['Age_Buckets_Factorized'] = pd.factorize(d.clean.preprocess['Age_-_Buckets'], sort = True)[0]
d.clean.preprocess.drop('Age_-_Buckets', axis = 1, inplace = True)

Job_Group_Dict = {'Operators' : 0, 'Support' : 1, 
                  'Operations Supervisors' : 2, 'Professional' : 3, 
                  'Management' : 4, 'Executive' : 5}
d.clean.preprocess['Job_Group_Factorized'] = d.clean.preprocess['Job_Group'].map(Job_Group_Dict)
d.clean.preprocess.drop('Job_Group', axis = 1, inplace = True)

#One-hot-encode the rest of the categorical features - only run this for the subset being used
d.clean.preprocess = pd.get_dummies(d.clean.preprocess, columns = ['Job_Category_(Job_Class)', 'Gender', 'Region'
                                             , 'Country', 'Management_Level_(Picklist_Label)',
                                             'Compensation_Grade', 'Marital_Status', 'Location', 
                                             'Ethnicity'])

In [ ]:
'''
This data had employees all over the world and therefore 
salary info from many different countries.
Here we read in a dataframe containing exchange rates and make these conversions
(up to date from the day of building the model).
'''

#Read in conversions
exchange = pd.read_csv('Exchange Rates.csv')

#Create a new column that converts the "midpoint" feature to USD
d['Midpoint_USD'] = None
for row in range(1,d.shape[0]):
    if not pd.isna(d['Currency'][row]):
        country = d['Currency'][row]
        d['Midpoint_USD'][row] = d['Annualized_Mid_Point'][row]/exchange[exchange['Currency'] == country].iloc[0][1]
    
#Create a new column that calculates salary based on USD Midpoint and compa ratio
d['Salary'] = None
for row in range(1,d.shape[0]):
    if not pd.isna(d['Midpoint_USD'][row]):
        d['Salary'][row] = d['Midpoint_USD'][row]*d['Compa_Ratio'][row]

In [ ]:
#Read in data provided by company to map the cost of employee churn by compensation grade
churnCost = pd.read_csv('CompansationGrade_cost_map.csv')

#Add "cost if churn" column using salary and churn cost
d.clean['Cost_If_Churn'] = None
for row in range(d.clean.shape[0]):
    if (not pd.isna(d.clean['Salary'][row])) & (not pd.isna(d.clean['Compensation_Grade'][row])):
        compa = d.clean['Compensation_Grade'][row]
        d.clean['Cost_If_Churn'][row] = d.clean['Salary'][row]*churnCost[churnCost['Compensation Grade'] == compa].iloc[0][1] 

#Create final dataframe containing cost if churn and GID_Anonymized
cost_df = d.clean.copy()
cost_df.reset_index(inplace = True)
cost_df = cost_df[['GID_anonymized', 'Cost_If_Churn']]